In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import datasets

# download and apply the transform
train_data = datasets.FashionMNIST('/content/data', train=True, download=True, transform=transforms.ToTensor())
train_data = list(train_data)[:4000]

# splitting the dataset into train and validation sets
train_data, val_data = train_data[:3500], train_data[3500:]



# plot the first 10 images in the training data
for i, (img, label) in enumerate(train_data[:25]):
    plt.subplot(5, 5, i+1)
    plt.imshow(img[0,...], cmap="gray")
    print(label)

plt.show()

In [ ]:
# build custom softmax module
class Softmax(torch.nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(Softmax, self).__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)

    def forward(self, x):
        pred = self.linear(x)
        return pred

# call Softmax Classifier
model_softmax = Softmax(784, 10)
model_softmax.state_dict()

# define loss, optimizier, and dataloader for train and validation sets
optimizer = torch.optim.SGD(model_softmax.parameters(), lr = 0.01)
criterion = torch.nn.CrossEntropyLoss()
batch_size = 16
train_loader = DataLoader(dataset = train_data, batch_size = batch_size)
val_loader = DataLoader(dataset = val_data, batch_size = batch_size)

epochs = 200
Loss = []
acc = []
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model_softmax(images.view(-1, 28*28))
        loss = criterion(outputs, labels)
        # Loss.append(loss.item())
        loss.backward()
        optimizer.step()
    Loss.append(loss.item())
    correct = 0
    for images, labels in val_loader:
        outputs = model_softmax(images.view(-1, 28*28))
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    accuracy = 100 * (correct.item()) / len(val_data)
    acc.append(accuracy)
    if epoch % 10 == 0:
        print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

plt.plot(Loss)
plt.xlabel("no. of epochs")
plt.ylabel("total loss")
plt.show()

plt.plot(acc)
plt.xlabel("no. of epochs")
plt.ylabel("total accuracy")
plt.show()

In [ ]:
outputs = model_softmax(images.view(-1, 28*28))
# plt.imshow(images[0,0,...].detach().numpy(), cmap='gray')
# plt.show()
# print(outputs[0])
for i in range(images.shape[0]):
    plt.subplot(images.shape[0], 1, i+1)
    plt.imshow(images[i,0,...].detach().numpy(), cmap='gray')
    print(outputs[i].argmax().item())

plt.show()